# Daily workflow - troselab (DRAFT)

`TR TODO: Jupyter ipywidget GUI`

(check Daniel's ipysheet mouse entry GUI for inspiration)

1. Upload to analysis server
    1. automatic (rcopy with file consolidation)?
    2. WinSCP?

    
2. Session ingest - Have this script parse the user folder. 
    1. retrieve login name 
    2. parse user storage folder for SessionIDs, ScanIDs
    3. intersect with 'Session' tables
    4. FIRST STEP: session ingestion of new data 
        1. have ipywidget with list of data to be ingested with checkbox 
        2. have dropdowns next to SessionsFetch that expose 'Project', 'Anatomical Location', 'Equipment' (NOTE: have me or admin accept new entries? Or user?) - have a user default
        3. have central 'START' button
    5. OPTIONAL: Have a notebook section to enter and insert Session Notes
    6. OPTIONAL: Submit Sessions to pyrat. Not sure if necessary.


3. Scan ingest / 1st stage s2p processing: 
    1. have another list with checkboxes (default checled) - now with all first stage scans of the newly ingested sessions. Have dropdown boxes with s2p parameteres. User default is selected. Followed by 'START'
    2. TODO: suppress s2p output, just have progress bars
    3. OPTIONAL: Have a notebook section to enter and insert Scan Notes
    4. OPTIONAL: Submit sessions, scans and Notes to RSPACE! Upload s2p average image, max image, Source image and Rastermap 
    

4. Scan Curation
    1. DIFFERENT NOTEBOOK (or different Notebook section)
    2. Make intersection of folder with database
        1. list _all_ scans
        2. have all non-curated red - the others green
        3. bhave non-processed scans grey
        4. have 'CURATE LOCALLY WITH s2P' buttons behind all scans. This button should spawn a local suite2p GUI which directly loads the respective suite2p stats.npy file from the server. Folder settings for server need to be set in datajoint.json file.

5. SERVER: Database backup and Data folder backup
    1. Have cronjob backup database to ana2 and isilon (midnight, incremental - only add, no delete. Increment ID to be reversible?)
    2. Have cronjob backup data folders to ana2 and isilon (midnight, incremental - only add, no delete)

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [12]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery 
import datajoint as dj
import ipywidgets as widgets
from natsort import natsorted, ns
import re

In [13]:
db_prefix = dj.config['custom'].get('database.prefix')
print('database Prefix:', db_prefix)

database Prefix: tobiasr_


## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [14]:
from adamacs.utility import *
from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment
import ipywidgets as widgets

Assign easy names for relevant tables

In [15]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

## 1. Data upload
(not implemented currently)

## 2. Session ingest

In [68]:
def select_sessions(SessionDirA, SessionDirB, Project, Recording_Location, Equipment, s2pparm):
    # Define the widgets
    session_dropdowns = []
    session_checkboxes = []
    for session_list in SessionDirB:
        # Dropdowns for Project, Recording Location, Equipment
        project_dropdown = widgets.Dropdown(options=Project, value=Project[0])
        location_dropdown = widgets.Dropdown(options=Recording_Location, value=Recording_Location[0])
        equipment_dropdown = widgets.Dropdown(options=Equipment, value=Equipment[0])
        s2pparms_dropdown = widgets.Dropdown(options=s2pparm, value=s2pparm[0])
        session_dropdowns.append((project_dropdown, location_dropdown, equipment_dropdown, s2pparms_dropdown))
        # Checkbox for Process?
        session_checkbox = widgets.Checkbox(description='Process?', layout=widgets.Layout(width='auto'))
        session_checkboxes.append(session_checkbox)

    # Display the widgets
    output = widgets.Output()

    with output:
        # Display the Sessions labels and associated dropdowns and checkboxes
        hbox_list = []
        for i, session_list in enumerate(SessionDirB):
            # Create an HBox to hold the label and associated dropdowns and checkbox
            hbox = widgets.HBox()
            hbox.children = [
                widgets.Label(value=session_list + ':', layout=widgets.Layout(width='1500px')), 
                session_dropdowns[i][0],
                session_dropdowns[i][1],
                session_dropdowns[i][2],
                session_dropdowns[i][3],
                session_checkboxes[i]
            ]
            if session_list in unique_directory_strings(SessionDirA, SessionDirB):
                hbox.children[0].value = '* ' + session_list + ':' 
            hbox_list.append(hbox)

        vbox = widgets.VBox(hbox_list, layout=widgets.Layout(flex='0 1 auto', overflow_y='scroll'))
        # Display the commit button
        commit_button = widgets.Button(description='Commit', layout=widgets.Layout(width='auto'))
        display(vbox, commit_button)

        # Define the callback function for the commit button
        def commit_button_clicked(b):
            selected_sessions = [SessionDirB[i] for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_projects = [session_dropdowns[i][0].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_locations = [session_dropdowns[i][1].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_equipment = [session_dropdowns[i][2].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            selected_s2pparms = [session_dropdowns[i][3].value for i in range(len(SessionDirB)) if session_checkboxes[i].value]
            output.clear_output()
            with output:
                print('Selected Sessions:', selected_sessions)
                print('Selected Projects:', selected_projects)
                print('Selected Recording Locations:', selected_locations)
                print('Selected Equipment:', selected_equipment)
                print('Selected S2PParms:', selected_s2pparms)
            return [selected_sessions, selected_projects, selected_locations, selected_equipment, selected_s2pparms]

        # Attach the callback function to the commit button
        commit_button.on_click(commit_button_clicked)

    # Display the output
    display(output)


def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    match = re.search(r'sess\S+', string)
    result = match.group(0)[0:] if match else 'No match found.'
    return result

def get_scan_key_from_dir(string):
    match = re.search(r'scan\S+_', string)
    result = match.group(0)[0:-1] if match else 'No match found.'
    return result

def unique_directory_strings(dirs1, dirs2):
    set1 = set(dirs1)
    set2 = set(dirs2)
    common_dirs = list(set1.intersection(set2))
    unique_dirs = list(set(set1.union(set2)) - set(common_dirs))
    return unique_dirs


In [44]:
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')
s2pparm = ['a', 'b', 'c' ]

dirs_ingested = session.SessionDirectory.fetch('session_dir')
SessionsDirArrayingested = get_session_dir_key_from_dir(dirs_ingested)
ScanDirArrayingested = get_scan_dir_key_from_dir(dirs_ingested)

dataroot = dj.config['custom']['exp_root_data_dir'][0]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and 'sess' in d]

sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)
# SessionsArrayRootData = get_session_key_from_dir(sorted_dirs_root)
# ScanArrayRootData = get_scan_key_from_dir(sorted_dirs_root)

# SessionsArray = session.Session().fetch('session_id')

In [11]:
print("\033[1m" + 'Select Project, Location, Setup, s2p parm' + "\033[0m" )
selected_data = select_sessions(SessionsDirArrayingested, sorted_dirs_root, Project, Recording_Location, Equipment, s2pparm)
# print(selected_data)

Select Project, Location, Setup, s2p parm


Output()

### 2.1 Session notes ingest

## 3. Scan ingest / s2p processing 1st pass

In [ ]:
def select_scans(SessionsArray, Project, Recording_Location, Equipment):
    # Define the widgets
    session_dropdowns = []
    session_checkboxes = []
    for session_list in SessionsArray:
        # Dropdowns for Project, Recording Location, Equipment
        project_dropdown = widgets.Dropdown(options=Project, value=Project[0])
        location_dropdown = widgets.Dropdown(options=Recording_Location, value=Recording_Location[0])
        equipment_dropdown = widgets.Dropdown(options=Equipment, value=Equipment[0])
        session_dropdowns.append((project_dropdown, location_dropdown, equipment_dropdown))
        # Checkbox for Process?
        session_checkbox = widgets.Checkbox(description='Process?', layout=widgets.Layout(width='auto'))
        session_checkboxes.append(session_checkbox)

    # Display the widgets
    output = widgets.Output()

    with output:
        # Display the Sessions labels and associated dropdowns and checkboxes
        hbox_list = []
        for i, session_list in enumerate(SessionsArray):
            # Create an HBox to hold the label and associated dropdowns and checkbox
            hbox = widgets.HBox()
            hbox.children = [
                widgets.Label(value=session_list + ':', layout=widgets.Layout(width='850px', font_weight='bold')),
                session_dropdowns[i][0],
                session_dropdowns[i][1],
                session_dropdowns[i][2],
                session_checkboxes[i]
            ]
            hbox_list.append(hbox)

        vbox = widgets.VBox(hbox_list, layout=widgets.Layout(flex='auto auto auto', overflow_y='scroll'))
        # Display the commit button
        commit_button = widgets.Button(description='Commit', layout=widgets.Layout(width='auto'))
        display(vbox, commit_button)

        # Define the callback function for the commit button
        def commit_button_clicked(b):
            selected_sessions = [SessionsArray[i] for i in range(len(SessionsArray)) if session_checkboxes[i].value]
            selected_projects = [session_dropdowns[i][0].value for i in range(len(SessionsArray)) if session_checkboxes[i].value]
            selected_locations = [session_dropdowns[i][1].value for i in range(len(SessionsArray)) if session_checkboxes[i].value]
            selected_equipment = [session_dropdowns[i][2].value for i in range(len(SessionsArray)) if session_checkboxes[i].value]
            output.clear_output()
            with output:
                print('Selected Sessions:', selected_sessions)
                print('Selected Projects:', selected_projects)
                print('Selected Recording Locations:', selected_locations)
                print('Selected Equipment:', selected_equipment)
            return [selected_sessions, selected_projects, selected_locations, selected_equipment]

        # Attach the callback function to the commit button
        commit_button.on_click(commit_button_clicked)

    # Display the output
    display(output)
